In [1]:
import pandas as pd
import math
import random

In [2]:
random.seed(0) # set seed to a constant so that same set of random numbers are generated each time this program is run
csv_input = '/Users/manu/Desktop/PS Project Files copy/kilnradiation.csv'
csv_output = '/Users/manu/Desktop/PS Project Files copy/heatloss_output.csv'
diameter_m = 4.75
interval_m = 1 # distance between consecutive temperature readings
temp_ambient_c = 29
clinker_rate = 290000 # Kg per hour

temp_ambient_k = temp_ambient_c + 273
area = math.pi * diameter_m * interval_m # pi * Diameter * length

def radiation(df, tempcol='temp(K)'):
    return 4.38 * 10**-8 * (df[tempcol]**4 - temp_ambient_k**4) * area

def convection(df, tempcol='temp(K)'):
    return 80.33 * (((df[tempcol]+temp_ambient_k)/2)**-0.724 * (df[tempcol]-temp_ambient_k)**1.333) * area

In [3]:
df = pd.read_csv(csv_input, header = None, names = ['length', 'temp(C)'])
df['temp(K)'] = df['temp(C)'] + 273
df['radiation'] = radiation(df)
df['convection'] = convection(df)
df['loss'] = (df['radiation'] + df['convection'])/clinker_rate

FileNotFoundError: [Errno 2] File /Users/manu/Desktop/PS Project Files copy/kilnradiation.csv does not exist: '/Users/manu/Desktop/PS Project Files copy/kilnradiation.csv'

In [ ]:
df.plot.scatter('length', 'loss',c = 'loss', cmap='hot_r')

In [ ]:
df.plot.scatter('length', 'length',c = 'loss', cmap='hot_r')

In [ ]:
df.style.hide_index().background_gradient(cmap='hot_r')

In [ ]:
loss_total = df['loss'].sum()
print(f'Total heat loss observed initially: {loss_total} KCal per Kg clinker')
df.to_csv(csv_output, index=False)

# How much heat can be saved?

### Outliers(ununsually high values) are detected using IQR as shown [here](https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)

In [ ]:
Q1 = df.loss.quantile(0.25)
Q3 = df.loss.quantile(0.75)
IQR = Q3 - Q1
upper_whisker = Q3 + 1.5 * IQR
lower_whisker = Q1 - 1.5 * IQR

def clean_outliers(loss):
    if loss > upper_whisker:
        return random.uniform(lower_whisker, upper_whisker)
    else:
        return loss

# replace unusually high values with random values that falls in normal temperature range
df['loss'] = df['loss'].map(clean_outliers)

In [ ]:
df.plot.scatter('length', 'loss',c = 'loss', cmap='hot_r')

In [ ]:
df.plot.scatter('length', 'length',c = 'loss', cmap='hot_r')

In [ ]:
df.style.hide_index().background_gradient(cmap='hot_r')

In [ ]:
loss_total_new = df.loss.sum()
savings = loss_total - loss_total_new

In [ ]:
print(f'about {savings:.3f} KCal can be saved for each Kg clinker produced')

In [ ]:
:.2f